In [1]:
import getpass

import fsspec
import pandas as pd

In [2]:
import barqs
import fasta
import fastq

In [3]:
# _password = getpass.getpass(prompt="Enter password for ssh host: ")
# fs = fsspec.filesystem("ssh", host="gesu.nygenome.org", username="collinsd", password=_password)

In [4]:
barcode_size = 16
umi_size = 12

In [6]:
raw_features = [
    "TGGTATCAGTCTCGT",
    "ACGTAGTGCACTAGA",
    "ATTGCCTCAGGGTTT",
    "AGAGCGACAGAATAC",
    "TGAGAGAAGGGACTA",
    "GCATCTCTCAAGCAA",
    "CACCCAATAGTAAGG",
    "TTAACCACTGTGTAC",
    "TCGTTATAACATCTG",
    "GCAGTTAAAGTCGGG",
    "AGAATAGGAGCCGCA",
    "ATACACAGTTTACCG",
    "CATCTAGACCTAACT",
    "GAGATACTGCCATAG",
]

In [7]:
features = [
    (seq, seq) for seq in raw_features
]

In [8]:
feature_map = {seq: name for name, seq in features}

In [9]:
# read1_file = fs.open("/brahms/kowalskim/data/hsc/multiome_072924/fastq/Multiome-HTO_S5_L001_R1_001.fastq.gz")
read1_file = "/Users/dcollins/workspace/data/jyun/HTO_R1.fastq.gz"
read1 = fastq.load(read1_file)

In [10]:
# read2_file = fs.open("/brahms/kowalskim/data/hsc/multiome_072924/fastq/Multiome-HTO_S5_L001_R2_001.fastq.gz")
read2_file = "/Users/dcollins/workspace/data/jyun/HTO_R2.fastq.gz"
read2 = fastq.load(read2_file)

In [11]:
identifiers = list(
    barqs.extract(read, umi_size=umi_size, barcode_size=barcode_size)
    for read in read1
)

In [12]:
reads = list(
    barqs.tag(read, barcode, umi, trim = False)
    for read, (barcode, umi) in zip(read2, identifiers)
)

In [13]:
trimmed_reads = list(
    barqs.trim_by_index(
        read, 
        feature_lookup=feature_map,
        region=(0, 15)
    ) 
    for read in reads
)

In [14]:
observed_features = list(barqs.filter_duplicates(trimmed_reads))

In [15]:
counts = barqs.quantify(observed_features, features)

In [23]:
counts_df = pd.DataFrame(counts).fillna(0).T

In [26]:
counts_df.to_csv("/Users/dcollins/workspace/data/jyun/HTO_counts.csv")